In [53]:
import json
import os
import pandas as pd

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [54]:
# Reading in results
### bbox format: (x1, y1, x2, y2)

with open(os.path.join("./results", "data_dict.json"), "r") as f:
    res_obj = json.load(f)

In [55]:
# Reading in ground truth
### bbox format: (x1, y1, w, h)

dir_name = "c1_split"

with open(os.path.join("./", dir_name, "test/result.json"), "r") as f:
    coco_gt_obj = json.load(f)

In [56]:
res_obj[0]

{'file_name': '/home/dtron2_user/ls_dtron2_full/model/c1_split/test/images/7eXDUz317futphPZ-loxi_ss.png',
 'height': 1080,
 'width': 1920,
 'annotations': [{'iscrowd': 0,
   'bbox': [0.0, 1021.9604, 1908.5348, 1080.0],
   'category_id': 0,
   'bbox_mode': 0},
  {'iscrowd': 0,
   'bbox': [780.5998, 523.4848, 1167.5743, 1004.3765],
   'category_id': 0,
   'bbox_mode': 0}]}

In [57]:
coco_gt_obj["annotations"][0]

{'id': 112335,
 'image_id': 8509,
 'category_id': 0,
 'segmentation': [],
 'bbox': [0.0, 2.958904109589041, 1920.0, 48.515625],
 'ignore': 0,
 'iscrowd': 0,
 'area': 93150.0}

In [58]:
# Group By Image

imgs_gt_pred_list = {}

tot_num_pred = 0
tot_num_gt = 0

for img in coco_gt_obj["images"]:
    fn1 = img["file_name"][2:]
    img_id = img["id"]
    
    gt_list = []

    for ann1 in coco_gt_obj["annotations"]:
        if img_id == ann1["image_id"]:
            x1 = ann1["bbox"][0]
            y1 = ann1["bbox"][1]
            w = ann1["bbox"][2]
            h = ann1["bbox"][3]
            gt_list.append([x1, y1, x1 + w, y1 + h])
            tot_num_gt += 1
    
    pred_list = []
    
    for res in res_obj:
        fn2 = os.path.basename(res["file_name"])
        if fn1 == fn2:
            for ann2 in res["annotations"]:
                pred_list.append(ann2["bbox"])
                tot_num_pred += 1
    
    imgs_gt_pred_list[fn1] = [gt_list, pred_list]
            



In [59]:
print(tot_num_pred)
print(tot_num_gt)

2633
1035


In [60]:
keys = list(imgs_gt_pred_list.keys())


In [61]:
def calculate_iou(boxA, boxB):
    
    # Determine the coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # Compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)
    
    # Compute the area of both the prediction and true bounding boxes
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    
    # Compute the area of union
    unionArea = boxAArea + boxBArea - interArea
    
    # Compute the Intersection over Union by dividing the intersection area by the union area
    iou = interArea / unionArea
    
    return iou

In [62]:
def calculate_int(boxA, boxB):
    
    # Determine the coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # Compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)
    
    return interArea

In [63]:
thold = .5

In [64]:
iou_scores = []

for key in keys:
    gts = imgs_gt_pred_list[key][0]
    preds = imgs_gt_pred_list[key][1]
    for i, pred in enumerate(preds):
        for j, gt in enumerate(gts):
            iou = calculate_iou(pred, gt)
            iou_scores.append([key, i, j, iou])
                
iou_scores_df = pd.DataFrame(iou_scores, columns=["fn", "pred_ind", "gt_ind", "iou"])

In [65]:
iou_scores_df

,fn,pred_ind,gt_ind,iou
0,qKWb4KrB2044SPm5-autoads_ss.png,0,0,0.792432
1,qKWb4KrB2044SPm5-autoads_ss.png,1,0,0.000000
2,kDmUDWTi1Ybj56Gd-dystel_ss.png,0,0,0.903051
3,kDmUDWTi1Ybj56Gd-dystel_ss.png,1,0,0.000000
4,kDmUDWTi1Ybj56Gd-dystel_ss.png,2,0,0.000000
...,...,...,...,...
3395,DsP9iPpWwZRHOR6L-nordictrack_ss.png,1,1,0.000000
3396,DsP9iPpWwZRHOR6L-nordictrack_ss.png,1,2,0.000000
3397,DsP9iPpWwZRHOR6L-nordictrack_ss.png,2,0,0.000000
3398,DsP9iPpWwZRHOR6L-nordictrack_ss.png,2,1,0.403289


In [66]:
iou_scores_df_g = iou_scores_df.groupby('fn')

In [67]:
iou_full_g_dfs = []
iou_corr_g_dfs = []
iou_dang_g_pred_dfs = []
iou_dang_g_gt_dfs = []
conf_mat = []

img_ctr = 0
pred_gtr_gt_list = []
gt_gtr_pred_list = []

for name, group in iou_scores_df_g:
    
    print(name)
    
    df_subset = group[["fn", "pred_ind", "gt_ind", "iou"]]
    df_subset = df_subset.sort_values(by="iou", ascending=False)
    df_subset = df_subset.reset_index(drop=True)
    
    iou_full_g_dfs.append(df_subset)
    tp = 0
    
    gt_list = list(df_subset["gt_ind"])
    pred_list = list(df_subset["pred_ind"])
        
    gt_set = list(set(gt_list))
    pred_set = list(set(pred_list))
    
    num_gt = len(gt_set)
    num_pred = len(pred_set)
    
    data_corr = []
    data_pred_dang = []
    data_gt_dang = []
    
    rem_flag = 0
    working_set = None
    
    # Number of predicted >= number of ground truths 
    if num_pred >= num_gt:
        pred_gtr_gt_list.append(img_ctr)
        working_set = pred_set
        rem_flag = 1
        
    # Number of ground truths > number of predicted    
    elif num_gt > num_pred:
        gt_gtr_pred_list.append(img_ctr)
        working_set = gt_set
        rem_flag = 2

    for i, row in df_subset.iterrows():
        
        if rem_flag == 1:
            if row["pred_ind"] in pred_set and row["iou"] > thold:
                data_corr.append(row)
                pred_set.remove(row["pred_ind"])
                tp += 1
            elif row["pred_ind"] in pred_set:
                data_pred_dang.append(row)
                pred_set.remove(row["pred_ind"])
            elif not pred_set:
                break
        elif rem_flag == 2:
            if row["gt_ind"] in gt_set and row["iou"] > thold:
                data_corr.append(row)
                gt_set.remove(row["gt_ind"])
                tp += 1
            elif row["gt_ind"] in gt_set:
                data_gt_dang.append(row)
                gt_set.remove(row["gt_ind"])
            elif not gt_set:
                break

    iou_corr_g_dfs.append(pd.DataFrame(data_corr))
    iou_dang_g_pred_dfs.append(pd.DataFrame(data_pred_dang))
    iou_dang_g_gt_dfs.append(pd.DataFrame(data_gt_dang))    
    
    fp = num_pred - tp
    fn = num_gt - tp
    
    conf_mat.append([tp, fp, fn])
    
    img_ctr += 1

0002HSj7V4ZbNIO4-publimaison_ss.png
048KxK73nIwwLoD0-mbbfstyle_ss.png
05rfcCBbt5EBAK4D-fultoncountyoh_ss.png
06aaCGuceDo5T5rC-prototypehubs_ss.png
0FX4MgBuXfPglIaX-bakerbotts_ss.png
0OB1J4nigM9MWAwT-teracoin_ss.png
0dBjOH00xLej6fhw-cases_ss.png
0eWNkcHSVk9SoaKG-villageinn_ss.png
0fOtTqW4ZNUX0lmU-domainecarneros_ss.png
0iuYnYW3EZO0NswG-bankbranchlocator_ss.png
0uIdKB6z4i351YjD-habitat_ss.png
10MuBssQtkAKOZiM-romeohotel_ss.png
16Vwl7KPKnwl1jb3-probaseballinsider_ss.png
18YXLjdOoVPrOc36-ridesnowboards_ss.png
1AOxzmMdAlaZ3JMm-islandinstitute_ss.png
1Lk2W8cvej6M7T05-illow_ss.png
1WZhcLmmGrh61EId-cncf_ss.png
1XE3lK8oYoYvJALd-navexglobal_ss.png
1ZiquciDhHb3C9yV-e-money_ss.png
1ae0MSaSapfs17ME-amshq_ss.png
1dfxPDeYu4w8XIiQ-iest_ss.png
1j9MviDD5K4dUFBU-dailylocal_ss.png
1z6MUji8nJGEWKaS-flyembraer_ss.png
1zZeWChna1qRzX8e-wayve_ss.png
20Z3M4KMAFTeKFUY-effectivestuffs_ss.png
214RW1kok7D6BoQW-pokerth_ss.png
26OSiDawubYW7Mr3-elpassion_ss.png
27LvAPm2f6K0kPua-jhapatechnical_ss.png
2De3Rd7lRHfHpBn2-u

In [68]:
tp_tot = 0
fp_tot = 0
fn_tot = 0

for img in conf_mat:
    tp_tot = tp_tot + img[0]
    fp_tot = fp_tot + img[1]
    # Would be equivalent to correctly identifying areas where no object is present... so we set it to 0 because it is not relevant
    tn_tot = 0
    fn_tot = fn_tot + img[2]
    

In [69]:
print("tp_tot", tp_tot)
print("fp_tot", fp_tot)
print("tn_tot", tn_tot)
print("fn_tot", fn_tot)

tp_tot 829
fp_tot 1804
tn_tot 0
fn_tot 197


In [70]:
print(len(pred_gtr_gt_list))

826


In [71]:
print(pred_gtr_gt_list)
print(gt_gtr_pred_list)

[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229,

In [72]:
voi = 5

In [73]:
iou_full_g_dfs[voi]

,fn,pred_ind,gt_ind,iou
0,0OB1J4nigM9MWAwT-teracoin_ss.png,0,0,0.918273
1,0OB1J4nigM9MWAwT-teracoin_ss.png,1,0,0.132059
2,0OB1J4nigM9MWAwT-teracoin_ss.png,2,0,0.000000
3,0OB1J4nigM9MWAwT-teracoin_ss.png,3,0,0.000000


In [74]:
iou_corr_g_dfs[voi]

,fn,pred_ind,gt_ind,iou
0,0OB1J4nigM9MWAwT-teracoin_ss.png,0,0,0.918273


In [75]:
iou_dang_g_pred_dfs[voi]

,fn,pred_ind,gt_ind,iou
1,0OB1J4nigM9MWAwT-teracoin_ss.png,1,0,0.132059
2,0OB1J4nigM9MWAwT-teracoin_ss.png,2,0,0.000000
3,0OB1J4nigM9MWAwT-teracoin_ss.png,3,0,0.000000


In [76]:
iou_dang_g_gt_dfs[voi]

""


In [77]:
accuracy = (tp_tot + tn_tot) / (tp_tot + fp_tot + tn_tot + fn_tot)
precision = tp_tot/(tp_tot + fp_tot)
recall = tp_tot/(tp_tot + fn_tot)
f1_score = 2 * (precision * recall) / (precision + recall)
print("accuracy: ", accuracy)
print("precision: ", precision)
print("recall: ", recall)
print("f1_score: ", f1_score)

accuracy:  0.2929328621908127
precision:  0.31484998101025446
recall:  0.8079922027290448
f1_score:  0.45312927029242966


In [78]:
with open('./results/metrics.txt', 'w+') as file:
    file.write(f"accuracy: {accuracy}\n")
    file.write(f"precision: {precision}\n")
    file.write(f"recall: {recall}\n")
    file.write(f"f1_score: {f1_score}\n")
